In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3gLnwjHQsdkzqoJ8acE'

In [3]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [4]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)
df_input['Part Code'] = df_input['Part Code'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：658



,No,Brand,Product Group,Part Number,Remanufactured,Status,Aditional Description,GTIN/EAN,OEM,Trade Number,Mandatory Material Certification,Packing Unit,Quantity Per Packaging Unit,Picture,Url,Json_Src,Json_Information,Json_Criteria,Part Code
0,1,VALEO,Wiper Blade,566001,False,Normal,EVERGUARD SILICONE SINGLE,3276425660014,,VSF35,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225694837
1,2,VALEO,Wiper Blade,566002,False,Normal,EVERGUARD SILICONE SINGLE,3276425660021,,VSF38,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225695203
2,3,VALEO,Wiper Blade,566003,False,Normal,EVERGUARD SILICONE SINGLE,3276425660038,,VSF40,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225695424
3,4,VALEO,Wiper Blade,566004,False,Normal,EVERGUARD SILICONE SINGLE,3276425660045,,VSF43,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225697032
4,5,VALEO,Wiper Blade,566005,False,Normal,EVERGUARD SILICONE SINGLE,3276425660052,,VSF45,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225697307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,654,VALEO,Wiper Blade,628652,False,Normal,OPTIBLADE TRUCKS,3276426286527,7482559001,,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...",{},"{""0"": {""Quantity"": ""1""}, ""1"": {""Length-1"": ""65...",815753228
654,655,VALEO,Wiper Blade,628700,False,Normal,OPTIBLADE TRUCKS,3276426287005,,,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Technical Information"": ""Product may d...","{""0"": {""Quantity"": ""1""}, ""1"": {""Length-1"": ""70...",13712148
655,656,VALEO,Wiper Blade,628701,False,Normal,OPTIBLADE TRUCKS,3276426287012,,,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Technical Information"": ""Product may d...","{""0"": {""Quantity"": ""1""}, ""1"": {""Length-1"": ""70...",13712172
656,657,VALEO,Wiper Blade,628702,False,Normal,OPTIBLADE TRUCKS,3276426287029,2371926;2753173,,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,{},{},"{""0"": {""Wiper blade type"": ""Beam wiper blade""}...",815758583


In [5]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                                                  1
 Brand                                                                           VALEO
 Product Group                                                             Wiper Blade
 Part Number                                                                    566001
 Remanufactured                                                                  False
 Status                                                                         Normal
 Aditional Description                                       EVERGUARD SILICONE SINGLE
 GTIN/EAN                                                                3276425660014
 OEM                                                                                  
 Trade Number                                                                    VSF35
 Mandatory Material Certification                                                False
 Packing Unit                  

In [6]:
data = {'getArticleLinkedAllLinkingTargetManufacturer2': {'articleCountry': 'DE',
                                                          'articleId': crawler_series['Part Code'],
                                                          'country': 'DE',
                                                          'countryGroupFlag': False,
                                                          'linkingTargetType': 'VLOB',
                                                          'provider': 23365}}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'x-api-key': api}),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
            elif dict_['status'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp

1


<Response [200]>

In [7]:
if type(dict_['data']) == str and dict_['data'].strip() == '':
    list_make = []
elif type(dict_['data']) == dict:
    list_make = dict_['data']['array']
else:
    print('fuck')

list_make

[{'manuId': 35, 'manuName': 'FIAT'},
 {'manuId': 36, 'manuName': 'FORD'},
 {'manuId': 63, 'manuName': 'LADA'},
 {'manuId': 64, 'manuName': 'LANCIA'},
 {'manuId': 80, 'manuName': 'NISSAN'},
 {'manuId': 84, 'manuName': 'OPEL'},
 {'manuId': 93, 'manuName': 'RENAULT'},
 {'manuId': 138, 'manuName': 'CHEVROLET'},
 {'manuId': 183, 'manuName': 'HYUNDAI'},
 {'manuId': 1138, 'manuName': 'SMART'},
 {'manuId': 3854, 'manuName': 'ABARTH'}]

In [8]:
list_dict_correct.extend([{'Part Code': crawler_series['Part Code'],
                           'Make': make['manuName'].strip(),
                           'Make Code': make['manuId'],
                           'JOIN_PCMC': f'''{crawler_series['Part Code']};{make['manuId']}''',
                           'Url': crawler_series['Url']} for make in list_make])

list_dict_correct

[{'Part Code': 1225694837,
  'Make': 'FIAT',
  'Make Code': 35,
  'JOIN_PCMC': '1225694837;35',
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/566001/detail?groups=298'},
 {'Part Code': 1225694837,
  'Make': 'FORD',
  'Make Code': 36,
  'JOIN_PCMC': '1225694837;36',
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/566001/detail?groups=298'},
 {'Part Code': 1225694837,
  'Make': 'LADA',
  'Make Code': 63,
  'JOIN_PCMC': '1225694837;63',
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/566001/detail?groups=298'},
 {'Part Code': 1225694837,
  'Make': 'LANCIA',
  'Make Code': 64,
  'JOIN_PCMC': '1225694837;64',
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/566001/detail?groups=298'},
 {'Part Code': 1225694837,
  'Make': 'NISSAN',
  'Make Code': 80,
  'JOIN_PCMC': '1225694837;80',
  'Url': 'https://web.tecalliance.net/tecdoc/qa/parts/21/566001/detail?groups=298'},
 {'Part Code': 1225694837,
  'Make': 'OPEL',
  'Make Code': 84,
  'JOIN_PCMC': '1225694837;

In [9]:
crawler_status = True

crawler_status

True

In [10]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                                  1
 Brand                                                                           VALEO
 Product Group                                                             Wiper Blade
 Part Number                                                                    566001
 Remanufactured                                                                  False
 Status                                                                         Normal
 Aditional Description                                       EVERGUARD SILICONE SINGLE
 GTIN/EAN                                                                3276425660014
 OEM                                                                                  
 Trade Number                                                                    VSF35
 Mandatory Material Certification                                                False
 Packing Unit                              

In [11]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_make.xlsx', index=False)

df_correct

,Part Code,Make,Make Code,JOIN_PCMC,Url
0,1225694837,ABARTH,3854,1225694837;3854,https://web.tecalliance.net/tecdoc/qa/parts/21...
1,1225694837,CHEVROLET,138,1225694837;138,https://web.tecalliance.net/tecdoc/qa/parts/21...
2,1225694837,FIAT,35,1225694837;35,https://web.tecalliance.net/tecdoc/qa/parts/21...
3,1225694837,FORD,36,1225694837;36,https://web.tecalliance.net/tecdoc/qa/parts/21...
4,1225694837,HYUNDAI,183,1225694837;183,https://web.tecalliance.net/tecdoc/qa/parts/21...
5,1225694837,LADA,63,1225694837;63,https://web.tecalliance.net/tecdoc/qa/parts/21...
6,1225694837,LANCIA,64,1225694837;64,https://web.tecalliance.net/tecdoc/qa/parts/21...
7,1225694837,NISSAN,80,1225694837;80,https://web.tecalliance.net/tecdoc/qa/parts/21...
8,1225694837,OPEL,84,1225694837;84,https://web.tecalliance.net/tecdoc/qa/parts/21...
9,1225694837,RENAULT,93,1225694837;93,https://web.tecalliance.net/tecdoc/qa/parts/21...


In [12]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_make_error.xlsx', index=False)

df_error

,No,Brand,Product Group,Part Number,Remanufactured,Status,Aditional Description,GTIN/EAN,OEM,Trade Number,Mandatory Material Certification,Packing Unit,Quantity Per Packaging Unit,Picture,Url,Json_Src,Json_Information,Json_Criteria,Part Code,Check_Url
0,1,VALEO,Wiper Blade,566001,False,Normal,EVERGUARD SILICONE SINGLE,3276425660014,,VSF35,False,1,,,https://web.tecalliance.net/tecdoc/qa/parts/21...,"{""0"": ""https://digital-assets.tecalliance.serv...","{""0"": {""Installation Information"": ""To apply t...","{""0"": {""Wiper blade type"": ""Flat bar wiper bla...",1225694837,https://web.tecalliance.net/tecdoc/qa/parts/21...
